In [1]:
import pandas as pd

In [149]:
df = pd.read_excel('C:\\Users\\vinh.nt2\\Downloads\\Comment_Analysis\\Data\\JBAGY_OFFICIAL\\Sentiment_split\\JBAGY_OFFICIAL_Quan_short_nam_category_split.xlsx')

c:\Users\vinh.nt2\AppData\Local\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
c:\Users\vinh.nt2\AppData\Local\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


In [150]:
columns_to_drop = ['Unnamed: 0.1', 'Unnamed: 0.2', 'Unnamed: 0.3', 'Unnamed: 0']
df = df.drop(columns=columns_to_drop)
df

,idx,comment,rating_star,comment_eng,category_sentiment,sentiment
0,0,shop đóng gói cẩn thận,5,shop packed carefully,Đóng gói và giao hàng,0.970897
1,0,giao hàng nhanh,5,fast delivery,Đóng gói và giao hàng,0.990744
2,0,đóng gói hộp rất đẹp có thể đem tặng quà,5,Beautiful box packaging can be given as a gift,Trải nghiệm,0.999937
3,0,quần mặc vừa form mặc lên rất oke,5,The pants fit well and look great on,Sản phẩm và chất lượng,0.999763
4,1,Chất vải mềm đẹp đường may chắc chắn giá cả hợ...,5,"Soft, beautiful fabric, sturdy seams, reasonab...",Giá cả và giá trị,1.000000
...,...,...,...,...,...,...
2919,945,Ok,4,Ok,Trải nghiệm,0.933994
2920,946,Tốt,5,Good,Đóng gói và giao hàng,0.919636
2921,947,Đẹp nghiện mê mê,5,Beautiful and addictive,Trải nghiệm,0.983479
2922,948,Quá tuyệt vời,5,so awesome,Khác,0.861933


In [151]:
import nltk
from nltk.corpus import twitter_samples
import matplotlib.pyplot as plt
import random
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [1]:
def process_tweet(tweet):

    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and
                word not in string.punctuation):
            stem_word = stemmer.stem(word)
            tweets_clean.append(stem_word)

    return tweets_clean

In [153]:
comments = df['comment_eng']

In [154]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import gensim
from gensim import corpora
import pyLDAvis.gensim_models
from stopwords import get_stopwords
from collections import Counter

df_keyword = pd.DataFrame(columns=['id', 'keyword', 'freq'])

# Tạo n-grams
def generate_ngrams(tokens, n=2):
    return list(ngrams(tokens, n))


for idx, comment in comments.items():
    processed_comments = []
    processed_comments.append(process_tweet(comments[idx]))

    bigrams = [generate_ngrams(comment, 2) for comment in processed_comments]
    trigrams = [generate_ngrams(comment, 3) for comment in processed_comments]

    bigrams = [item for sublist in bigrams for item in sublist]
    trigrams = [item for sublist in trigrams for item in sublist]

    bigram_freq = Counter(bigrams)
    trigram_freq = Counter(trigrams)

    print(comment)
    print("Top bigrams:")
    keyword_b = ''
    for bigram, freq in bigram_freq.most_common(len(processed_comments[0])):
        keyword_b = ' '.join(bigram)
        print(f"{' '.join(bigram)}: {freq}")
        new_row = {'id': idx, 'keyword': keyword_b, 'freq': freq}
        df_keyword.loc[len(df_keyword)] = new_row

    print("\nTop trigrams:")
    keyword_b = ''
    for trigram, freq in trigram_freq.most_common(len(processed_comments[0])):
        keyword_b = ' '.join(trigram)
        print(f"{' '.join(trigram)}: {freq}")
        new_row = {'id': idx, 'keyword': keyword_b, 'freq': freq}
        df_keyword.loc[len(df_keyword)] = new_row

df_keyword

shop packed carefully
Top bigrams:
shop pack: 1
pack care: 1

Top trigrams:
shop pack care: 1
fast delivery
Top bigrams:
fast deliveri: 1

Top trigrams:
Beautiful box packaging can be given as a gift
Top bigrams:
beauti box: 1
box packag: 1
packag given: 1
given gift: 1

Top trigrams:
beauti box packag: 1
box packag given: 1
packag given gift: 1
The pants fit well and look great on
Top bigrams:
pant fit: 1
fit well: 1
well look: 1
look great: 1

Top trigrams:
pant fit well: 1
fit well look: 1
well look great: 1
Soft, beautiful fabric, sturdy seams, reasonable price, very classy in the box
Top bigrams:
soft beauti: 1
beauti fabric: 1
fabric sturdi: 1
sturdi seam: 1
seam reason: 1
reason price: 1
price classi: 1
classi box: 1

Top trigrams:
soft beauti fabric: 1
beauti fabric sturdi: 1
fabric sturdi seam: 1
sturdi seam reason: 1
seam reason price: 1
reason price classi: 1
price classi box: 1
Fast delivery shop
Top bigrams:
fast deliveri: 1
deliveri shop: 1

Top trigrams:
fast deliveri sh

,id,keyword,freq
0,0,shop pack,1
1,0,pack care,1
2,0,shop pack care,1
3,1,fast deliveri,1
4,2,beauti box,1
...,...,...,...
14416,2918,true descript,1
14417,2918,descript 0k,1
14418,2918,true descript 0k,1
14419,2921,beauti addict,1


In [155]:
df_freq = df_keyword.groupby(['keyword'], as_index=False).agg({'freq': 'sum'})

In [156]:
df_freq_sorted = df_freq.sort_values(by='freq', ascending=False)
df_freq_sorted

,keyword,freq
3422,fast deliveri,163
5808,nice pant,84
9145,support shop,45
4116,good qualiti,41
4106,good product,39
...,...,...
3814,futur use,1
3815,futur use quit,1
3816,g h,1
3817,g h j,1


In [157]:
dict = {}
for idx, row in df_freq.iterrows():
    dict[row[0]] = row[1]

C:\Users\vinh.nt2\AppData\Local\Temp\ipykernel_5368\4067640910.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dict[row[0]] = row[1]


In [158]:
for idx, row in df_keyword.iterrows():
    df_keyword.at[idx, 'freq'] = dict[row['keyword']]

In [159]:
df_keyword

,id,keyword,freq
0,0,shop pack,14
1,0,pack care,30
2,0,shop pack care,8
3,1,fast deliveri,163
4,2,beauti box,1
...,...,...,...
14416,2918,true descript,28
14417,2918,descript 0k,2
14418,2918,true descript 0k,2
14419,2921,beauti addict,1


In [2]:
df_keyword = pd.read_excel('C:\\Users\\vinh.nt2\\Downloads\\Comment_Analysis\\Data\\JBAGY_OFFICIAL\\Keyword\\keyword_extract\\JBAGY_OFFICIAL_Quan_short_nam_category_split.xlsx')

In [5]:
df_keyword.sort_values(by='freq', ascending=False)

,Unnamed: 0,id,keyword,freq
1035,1035,202,fast deliveri,163
11846,11846,2341,fast deliveri,163
12285,12285,2412,fast deliveri,163
4291,4291,865,fast deliveri,163
12242,12242,2403,fast deliveri,163
...,...,...,...,...
6326,6326,1327,shirt fast,1
6330,6330,1327,beauti shirt fast,1
6331,6331,1327,shirt fast deliveri,1
6333,6333,1327,deliveri enthusiast shop,1
